In [1]:
import os
import gc
import numpy as np

from scipy.optimize import curve_fit
from obspy import UTCDateTime
from pandas import DataFrame, date_range

import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append('../')

In [3]:
if os.uname().nodename == 'lighthouse':
    root_path = '/home/andbro/'
    data_path = '/home/andbro/kilauea-data/'
    archive_path = '/home/andbro/freenas/'
    bay_path = '/home/andbro/bay200/'
elif os.uname().nodename == 'kilauea':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'
elif os.uname().nodename == 'lin-ffb-01':
    root_path = '/home/brotzer/'
    data_path = '/import/kilauea-data/'
    archive_path = '/import/freenas-ffb-01-data/'
    bay_path = '/bay200/'

In [5]:
config = {}

config['path_to_data'] = data_path+"ids/data/"
config['path_to_data3'] = data_path+f"ids/data03/"
config['path_to_data7'] = data_path+f"ids/data07/"

config['path_to_outdata'] = data_path+"ids/data/"

config['path_to_figures'] = data_path+"ids/figures/"

# experiment with cameras pararell at different rings
config['date1'] = "2024-05-22"
config['date2'] = "2024-06-23"

# conversion from pixel to mm
config['conversion'] = {"XX":1.67e-3,
                        "03":5.3e-3,
                        "07":5.3e-3,
                       }


In [6]:
from functions.load_mlti import __load_mlti
from functions.get_mlti_intervals import __get_mlti_intervals
from functions.mlti_intervals_to_NaN import __mlti_intervals_to_NaN

In [7]:
mltiU = __load_mlti(config['date1'], config['date2'], "U", archive_path)
mltiU_t1, mltiU_t2 = __get_mlti_intervals(mltiU.time_utc)

mltiZ = __load_mlti(config['date1'], config['date2'], "Z", archive_path)
mltiZ_t1, mltiZ_t2 = __get_mlti_intervals(mltiZ.time_utc)

mltiV = __load_mlti(config['date1'], config['date2'], "V", archive_path)
mltiV_t1, mltiV_t2 = __get_mlti_intervals(mltiV.time_utc)


In [8]:
def __load_beam_wander_data(tbeg, tend, path_to_data):

    from obspy import UTCDateTime
    from datetime import date
    from pandas import read_pickle, concat, DataFrame, date_range, to_datetime

    t1 = date.fromisoformat(str(UTCDateTime(tbeg).date))
    t2 = date.fromisoformat(str((UTCDateTime(tend)).date))

    df = DataFrame()
    for dat in date_range(t1, t2):

        # print(str(dat)[:10])

        file = f"{str(dat)[:10].replace('-','')}.pkl"
        try:
            df0 = read_pickle(path_to_data+file)
            df = concat([df, df0])
        except:
            print(f"error for {file}")

    # remove NaN from time column
    df.dropna(subset=['time'], inplace=True)

    # reset the index column
    df.reset_index(inplace=True, drop=True)

    # add column for relative time in seconds
    df['time_sec'] = [UTCDateTime(_t) - UTCDateTime(df.time.iloc[0]) for _t in df.time]

    return df

In [9]:
df = __load_beam_wander_data(config['date1'], config['date2'], config['path_to_data'])

error for 20240612.pkl
error for 20240623.pkl


In [10]:
df3 = __load_beam_wander_data(config['date1'], config['date2'], config['path_to_data3'])

In [11]:
df7 = __load_beam_wander_data(config['date1'], config['date2'], config['path_to_data7'])

error for 20240623.pkl


### Remove MLTI times

In [12]:
def __pre_process(df0, mlti_t1, mlti_t2, conversion_factor):

    from functions.reduce import __reduce

    # local time to utc (! correct only in winter)
    df0['time_utc'] = [UTCDateTime(tl)-3600 for tl in df0.time]

    keys_to_convert = ["y_idx", "x_idx", "y", "x", "x_sig", "y_sig"]

    for k in keys_to_convert:
        df0[k] *= conversion_factor

    # turn times with MLTI to NaN
    df0 = __mlti_intervals_to_NaN(df0, "y_idx", "time_utc", mlti_t1, mlti_t2, t_offset_sec=180)
    df0 = __mlti_intervals_to_NaN(df0, "x_idx", "time_utc", mlti_t1, mlti_t2, t_offset_sec=180)

    # remove bad estimates
    df0 = df0[df0['y_sig_var'] != np.inf]
    df0 = df0[df0['x_sig_var'] != np.inf]

    # when intensity is super low, there is a black image
    df0 = df0[df0['amp'] > 20]

    # when intensity is super high or saturated, there is likely a MLTI boost
    df0 = df0[df0['amp'] < 255]

    # to micrometer
    df0['x_um_rel'] = __reduce(df0.x, 10) * 1e3
    df0['y_um_rel'] = __reduce(df0.y, 10) * 1e3

    df0['x_um_sig'] = df0.x_sig * 1e1
    df0['y_um_sig'] = df0.y_sig * 1e1

    # df0['x_um_sig'] = __reduce(df0.x_sig, 10) * 1e3
    # df0['y_um_sig'] = __reduce(df0.y_sig, 10) * 1e3

    df0.reset_index(inplace=True)

    return df0

In [13]:
df = __pre_process(df, mltiU_t1, mltiU_t2, config['conversion']["XX"])

In [14]:
df7 = __pre_process(df7, mltiV_t1, mltiV_t2, config['conversion']["07"])

In [15]:
df3 = __pre_process(df3, mltiZ_t1, mltiZ_t2, config['conversion']["03"])

In [16]:
# correct offset for df7
# df7['x_um_rel'] = [x - 900 if x > 500 else x for x in df7.x_um_rel]

In [17]:
df7['x_um_rel'] = [np.nan if x < 140 else j for x, j in zip(df7.index, df.x_um_rel)]
df7['y_um_rel'] = [np.nan if x < 140 else j for x, j in zip(df7.index, df.y_um_rel)]

In [18]:
from functions.load_lxx import __load_lxx
from functions.get_lxx_intervals import __get_lxx_intervals

lxx =__load_lxx(config['date1'], config['date2'], archive_path)
lxx_t1, lxx_t2 = __get_lxx_intervals(lxx.datetime)


### Plotting

In [22]:
def __makeplot_overview():

    Nrow, Ncol = 3, 1

    font = 12
    size = 2
    tscale = 1/86400

    fig, ax = plt.subplots(Nrow, Ncol, figsize=(12, 8), sharex=True)

    plt.subplots_adjust(hspace=0.1)

    for n, (_df, ring) in enumerate(zip([df, df3, df7], ["RU", "RZ", "RV"])):

        ax[n].fill_between(_df.time_sec*tscale, _df.x_um_rel-abs(_df.x_um_sig)/2, _df.x_um_rel+abs(_df.x_um_sig)/2,
                           color="tab:blue", alpha=0.3, interpolate=False
                          )
        ax[n].scatter(_df.time_sec*tscale, _df.x_um_rel, s=size, zorder=2, color="tab:blue", label=f"rel. X position ({ring})")

        ax[n].fill_between(_df.time_sec*tscale, _df.y_um_rel-abs(_df.y_um_sig)/2, _df.y_um_rel+abs(_df.y_um_sig)/2,
                           color="tab:red", alpha=0.3, interpolate=False
                          )
        ax[n].scatter(_df.time_sec*tscale, _df.y_um_rel, s=size, zorder=2, color="tab:red", label=f"rel. Y position ({ring})")
        # ax[n].set_ylim(1.58, 1.69)
        ax[n].set_ylabel(f"Displacement ($\mu$m)", fontsize=font)

        ax[n].legend(loc=8, ncol=2)

        # add maintencane periods
        ref_date = config['date1']
        for lx1, lx2 in zip(lxx_t1, lxx_t2):
            lx1_sec = lx1-UTCDateTime(ref_date)
            lx2_sec = lx2-UTCDateTime(ref_date)
            ax[n].fill_betweenx([-1000, 1000], lx1_sec*tscale, lx2_sec*tscale, color="yellow", alpha=0.5)

        y_min, y_max = min([_df.x_um_rel.min(), _df.y_um_rel.min()]), max([_df.x_um_rel.max(), _df.y_um_rel.max()])
        ax[n].set_ylim(y_min*1.2, y_max*1.2)


    for _k, ll in enumerate(['(a)', '(b)', '(c)']):
        ax[_k].text(.005, .97, ll, ha='left', va='top', transform=ax[_k].transAxes, fontsize=font+2)


    ax[Nrow-1].set_xlabel(f"Time (days) from {config['date1']}", fontsize=font)

    for _n in range(Nrow):
        ax[_n].grid(ls=":", zorder=0)
        # ax[_n].legend(loc=1)

    ax[1].set_ylim(top=150)

    plt.show();
    return fig

fig = __makeplot_overview();

fig.savefig(config['path_to_figures']+f"BeamWander_overview_{config['date1']}_{config['date2']}.png", format="png", dpi=150, bbox_inches='tight')
